In [1]:
# Importação de bibliotecas
import pandas as pd

from pendulum import now

In [2]:
# Objetos
fonte = 'obscana'
dataset_name = 'etanol'

raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Bronze

In [4]:
ano_inicio = now().year - 2
ano_fim = now().year - 1

url = f'https://observatoriodacana.com.br/historico-de-producao-e-moagem.php?idMn=31&tipoHistorico=2&acao=visualizar&idTabela=2492&produto=etanol_total&safraIni=1999%2F2000&safraFim={ano_inicio}%2F{ano_fim}&estado=RS%2CSC%2CPR%2CSP%2CRJ%2CMG%2CES%2CMS%2CMT%2CGO%2CDF%2CBA%2CSE%2CAL%2CPE%2CPB%2CRN%2CCE%2CPI%2CMA%2CTO%2CPA%2CAP%2CRO%2CAM%2CAC%2CRR'

tabela = (
    pd.read_html(f'{url}', thousands = ".", decimal = ",")[0]
    .to_html(f'{raiz}/bronze_{fonte}_{dataset_name}.html', index=False)
 )

### Prata

In [14]:
anos = (
    pd.read_html(f'{raiz}/bronze_{fonte}_{dataset_name}.html', encoding='utf-8')[0]
    .set_index('Estado')
    .rename(columns=lambda col: col[-4:])
    .columns
    .tolist()
)

In [26]:
df = (pd.read_html(f'{raiz}/bronze_{fonte}_{dataset_name}.html',
                   encoding='utf-8',
                   na_values=0
                   )[0]
      .query('not Estado.str.contains("Região.*|Brasil", case=False)')    
      .rename(columns=lambda col: col if col == 'Estado' else col[-4:])
      .merge(pd.read_parquet(f'{raiz}/prata_ibge_geocodigo.parquet')
              .filter(['UF-id', 'UF-nome'])
              .drop_duplicates(),
              how='left',
              left_on='Estado',
              right_on='UF-nome'
              )
      .melt(id_vars='UF-id',
            value_vars=anos,
            var_name='ano',
            value_name='safra_m3')
      .assign(**{
            'data': lambda df: pd.to_datetime(df['ano'], format='%Y') # Coluna "data" em datetime baseado na ISO 8601, convertido da coluna "D3C"
            })
      .drop(columns='ano')
      .dropna(subset='safra_m3')
      .convert_dtypes()
      .rename(columns={'UF-id':'uf'})
      .sort_values(by=['uf', 'data'])
      .set_index(['uf', 'data'])
      )

df.to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}.parquet')

### Ouro

Produção de Etanol ao todo por Estado (1000 m³)

Parâmetros
Teor (kg N/m³)

Cálculo: ((produção estadual * producao)*(parametro_X))*(1000/1000)

Brasil = mil t

NPK na forma do fertilizante orgânico vinhaça incorporado ao solo (t)

In [20]:
producao = 11.5

parametro_N = 0.433
parametro_P = 0.034
parametro_K = 2.206

In [27]:
df = (pd.read_parquet(f'{raiz}/prata_{fonte}_{dataset_name}.parquet')
      .assign(**{
          'N': lambda df: ((df['safra_m3']*producao)*parametro_N)*(1000/1000),
          'P': lambda df: ((df['safra_m3']*producao)*parametro_P)*(1000/1000),
          'K': lambda df: ((df['safra_m3']*producao)*parametro_K)*(1000/1000),
          'fonte': 'Fertilizante orgânico vinhaça',
          })
      .reset_index()
      .melt(id_vars=['uf', 'data', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor')
      .convert_dtypes()
      .set_index(['uf', 'data', 'fonte'])
)

In [31]:
df = (
    df
    .rename_axis(index={
        'uf':'state',
        'data': 'date',
        'fonte': 'label'
    })
    .rename(columns={
        'elemento':'nutrient',
        'valor':'value'
    })
)

df.to_csv(f'{raiz}/ouro_npk_fert_organico_vinhaca.csv')